# Multilayer Perceptron: Market  Timing of APPL stock with Hyper-parameter Tuning

Goal: Implement Keras Tuner in mlp model for stock timing.

**1. Data**

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

df = yf.download("AAPL", start="1980-01-01", end="2022-04-11")

df["Ret"] = df["Adj Close"].pct_change()
df.reset_index(inplace=True)

name = "Ret"

df.tail()

**Inputs and outputs**

In [ ]:
df["Ret25_i"] = df[name].rolling(25).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret60_i"] = df[name].rolling(60).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret90_i"] = df[name].rolling(90).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret120_i"] = df[name].rolling(120).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret240_i"] = df[name].rolling(240).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))

del df["Open"]
del df["Close"]
del df["High"]
del df["Low"]
del df["Volume"]
del df["Adj Close"]

df = df.dropna()
df.tail(10)

**Defining the output: Regression**

- Output is actually a continuous variable containing the returns of AAPL stock for some future time span.
- The model will aim to predict the return of AAPL stock over the next 25 days.

In [ ]:
df["Ret25"] = df["Ret25_i"].shift(-25)
df = df.dropna()
df.tail(10)

In [ ]:
df.describe()

**Train-Test samples and Scaling**

In [ ]:
ts = int(0.4 * len(df))  # Number of observations in the test sample
split_time = len(df) - ts  # From this data we are in the test sample
test_time = df.iloc[split_time:, 0:1].values  # Keep the test sample dates
Ret_vector = df.iloc[split_time:, 1:2].values
df.tail()

In [ ]:
from sklearn.model_selection import train_test_split

Xdf, ydf = df.iloc[:, 2:-1], df.iloc[:, -1]
X = Xdf.astype("float32")
y = ydf.astype("float32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=ts, shuffle=False
)  # It is important to keep "shuffle=False"
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

**2. Model and Training**

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from keras_tuner import HyperModel
import os
import matplotlib.pyplot as plt
from keras import backend as K

- Dropout: we are letting the tuner choose between a value for dropout of 0.2 or 0.3.

- Number of layers in the model: in this case, we will let the tuner choose between 1 and 5 hidden layers in the model. We do this using the hp.Int() method.

- Number of units in each hidden layer: the tuner will choose between 1 (min_value) and 25 (max_value) units in each layer, with a step size of 5.

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(1234)
val_split = 0.2


class MLP_model(HyperModel):
    def build(self, hp):
        # We define a constant activation function of ReLU form. We will not be tuning the activation functions
        act_fun = "relu"

        # We do ask the Keras Tuner to choose whether is best to have a dropout rate after each hidden layer of 0.2 or 0.3
        n_dropout = hp.Choice("n_dropout", values=[0.20, 0.30])

        model = tf.keras.models.Sequential()

        # Now, we will use a loop to let the tuner choose the number of layers that is best for the model between 1 and 5
        for i in range(1, hp.Int("num_layers", 1, 5)):
            # Within this loop, we will also ask the tuner to decide the optimal number of units that each of the selected layer should have.
            model.add(
                tf.keras.layers.Dense(
                    units=hp.Int(
                        "units_dense_" + str(i), min_value=1, max_value=25, step=5
                    ),
                    activation=act_fun,
                )
            )
            model.add(tf.keras.layers.Dropout(n_dropout))

        model.add(tf.keras.layers.Dense(units=1))

        # As was the case with the activation function, there is no tuning on the learning rate, nor the optimizer, although we could easily incorporate it.
        hp_lr = 1e-5
        adam = tf.keras.optimizers.Adam(learning_rate=hp_lr)
        model.compile(optimizer=adam, loss="mean_absolute_error")

        return model

- Hypermodel: we have already defined our hypermodel: MLP_model()

- Objective: what is the objective function to optimize? In this case we will minimize the loss function in the validation set.

- Other parameters: these include whether we overwrite results, fixing the random seed, or storage of results (convenient for large complex models).

In [ ]:
# First, we clear the session just to make sure our seeds are correctly working and replicability is achieved
K.clear_session()

# Then, we call the model and perform the tuning:
hypermodel = MLP_model()
tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective("val_loss", direction="min"),
    overwrite=True,
    max_epochs=30,
    seed=1234,
    directory=os.path.normpath("C:/"),
)

# Let's run the tuner! (Warning: this could take time)
tuner.search(X_train, y_train, validation_split=val_split)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_hps.values

In [ ]:
model = tuner.hypermodel.build(best_hps)
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", verbose=1, patience=20, restore_best_weights=True
)

# fit the model
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    verbose=2,
    callbacks=[es],
)

In [ ]:
model.summary()

**3. Financial performance of the model**

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
df_predictions = pd.DataFrame(
    {
        "Date": test_time.flatten(),
        "Pred": y_pred.flatten(),
        "Ret": (Ret_vector.flatten()),
    }
)
df_predictions.tail()

In [ ]:
df_predictions.Date = pd.to_datetime(df_predictions.Date, format="%YYYY-%mm-%dd")
df = df_predictions
df.tail()

Strategies:
- Buy and hold: simply buys the stock at the beginning of the test period and holds it until the end.

- Long-only: this strategy either goes long in the stock (if the predicted 25-day return is positive) or goes to cash (with a 0% return associated).

- Long/Short this strategy goes long (short) in the stock when the 25-day ahead return predicted by the model is positive (negative)

In [ ]:
df["Positions"] = np.where(df["Pred"] > 0, 1, -1)
df["Strat_ret"] = df["Positions"].shift(1) * df["Ret"]
df["Positions_L"] = df["Positions"].shift(1)
df["Positions_L"][df["Positions_L"] == -1] = 0
df["Strat_ret_L"] = df["Positions_L"] * df["Ret"]
df["CumRet"] = df["Strat_ret"].expanding().apply(lambda x: np.prod(1 + x) - 1)
df["CumRet_L"] = df["Strat_ret_L"].expanding().apply(lambda x: np.prod(1 + x) - 1)
df["bhRet"] = df["Ret"].expanding().apply(lambda x: np.prod(1 + x) - 1)

Final_Return_L = np.prod(1 + df["Strat_ret_L"]) - 1
Final_Return = np.prod(1 + df["Strat_ret"]) - 1
Buy_Return = np.prod(1 + df["Ret"]) - 1

print("Strat Return Long Only =", Final_Return_L * 100, "%")
print("Strat Return =", Final_Return * 100, "%")
print("Buy and Hold Return =", Buy_Return * 100, "%")

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.gca()
df.plot(x="Date", y="bhRet", label="Buy&Hold", ax=ax)
df.plot(x="Date", y="CumRet_L", label="Strat Only Long", ax=ax)
df.plot(x="Date", y="CumRet", label="Strat Long/Short", ax=ax)
plt.xlabel("date")
plt.ylabel("Cumulative Returns")
plt.grid()
plt.show()

df.describe()